### Ingestión del archivo "genre.csv"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run ../includes/common_functions

#### Paso 1 - Leer archvio CSV usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
genre_schema = StructType(fields = [
    StructField("genreId", IntegerType(), False),
    StructField("genreName", StringType(), True)
])

In [0]:
genre_df = spark.read \
    .option("header",True) \
    .schema(genre_schema)\
    .csv(f"{bronze_folder_path}/{v_file_date}/genre.csv")

#### Pase 2 - Cambiar el nombre de las columnas según lo requerido

In [0]:
genre_renamed_df = genre_df.withColumnRenamed("genreId", "genre_id").withColumnRenamed("genreName", "genre_name")


#### Paso 3 - Agregar la columna "ingestion_date" y "environment" al data frame

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
genre_final_df = add_ingestion_date(genre_renamed_df) \
                .withColumn("environment", lit(v_environment)) \
                .withColumn("file_date", lit(v_file_date))
#genre_renamed_df.withColumn("ingestion_date", current_timestamp()).withColumn("environment", lit("production"))

#### Paso 4 - Escribir datos en el DataLake en formato "Parquet"

In [0]:
genre_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.genres")
#genre_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.genres")
#genre_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/genres")

In [0]:
%sql
SELECT * FROM movie_silver.genres;

In [0]:
dbutils.notebook.exit("Success")